In [33]:
import requests, arcpy, datetime
import xml.etree.ElementTree as ET
from arcgis.gis import GIS

## On ArcGIS Online, loop through submissions from a webform (Survey123) to calculate iTree benefits
## These benefits can then be displayed on a ArcGIS Online Hub site app.

try:
	arcpy.env.overwriteOutput = True
	
	# fields in the schema
	fields = [
		'Email',
		'Address',
		'City',
		'State',
		'Zip',
		'County',
		'Nation',
		'DatePlanted',
		'ProjectYears',
		'MortalityRate',
		'TreeLocation_Long',
		'TreeLocation_Lat',
		'TreeAmount',
		'SpeciesCode_Key',
		'SpeciesCode_Value',
		'SpeciesOther',
		'SpeciesScientific',
		'DBH_CM',
		'TreeHeightMeter',
		'TreeCrownWidthMeter',
		'TreeCrownHeightMeter',
		'Condition',
		'CLE',
		'LandType',
		'Within60ftBuilding',
		'EnergyOnly',
		'HouseVintage',
		'DirectionDegree',
		'DistanceMeter',
		'DistanceFeet',
		'Heated',
		'AirConditioned',
		'QualityControl',
		'TreeLove',
		'TreeNotes',
		'Calc_RunoffAvoidedGallons',
		'Calc_PollRemoved',
		'Calc_PollRemovedValue',
		'created_user',
		'created_date',
		'last_edited_user',
		'last_edited_date',
		'OBJECTID',
		'RunoffAvoided',
		'RunoffAvoidedValue',
		'Interception',
		'PotentialEvaporation',
		'PotentialEvapotranspiration',
		'Evaporation',
		'Transpiration',
		'MbtuHeatShade',
		'MbtuHeatShadeValue',
		'MbtuHeatClimate',
		'MbtuHeatClimateValue',
		'MbtuWindHeat',
		'MbtuWindHeatValue',
		'KWhElectricityHeatShade',
		'KWhElectricityHeatShadeValue',
		'KWhElectricityHeatClimate',
		'KWhElectricityHeatClimateValue',
		'KWhElectricityHeatWind',
		'KWhElectricityHeatWindValue',
		'KWhElectricityCoolShade',
		'KWhElectricityCoolShadeValue',
		'KWhElectricityCoolClimate',
		'KWhElectricityCoolClimateValue',
		'KWhPrice',
		'ThermPrice',
		'PollRemoved_CO',
		'PollRemoved_COValue',
		'PollRemoved_NO2',
		'PollRemoved_NO2Value',
		'PollRemoved_SO2',
		'PollRemoved_SO2Value',
		'PollRemoved_O3',
		'PollRemoved_O3Value',
		'PollRemoved_PM25',
		'PollRemoved_PM25Value',
		'PollAvoided_CO',
		'PollAvoided_COValue',
		'PollAvoided_NO2',
		'PollAvoided_NO2Value',
		'PollAvoided_SO2',
		'PollAvoided_SO2Value',
		'PollAvoided_PM25',
		'PollAvoided_PM25Value',
		'PollAvoided_VOC',
		'PollAvoided_VOCValue',
		'Sequestered_CO2',
		'Sequestered_CO2Value',
		'Avoided_CO2',
		'Avoided_CO2Value',
		'CarbonStorage',
		'CarbonDioxideStorage',
		'CarbonDioxideStorageValue',
		'DryWeight'
	]
	
	# get the feature service from arcgis online
	g = GIS("https://www.arcgis.com","user","pass")
	
	## just for reference: getting by title is not as precise as by unique ID (see below)
	## fs = g.content.search(query="title:CommunityTrees", max_items=1)
	## url = fs[0].layers[0].url
	
	# get feature service by its unique ID ('Knox Co Community Trees' feature layer on ArcGIS Online)
	fs = g.content.get('893f6c29e8b64c66b39d9a3f9b1c8609')
	print(fs.title)
	arcgis_url = fs.url+'/0'
	print(arcgis_url)
	
	# loop through submitted trees in the resource layer
	with arcpy.da.UpdateCursor(arcgis_url, fields) as cursor:
		# calculate yesterday to find recent submissions
		today = datetime.datetime.today()
		yesterday = today - datetime.timedelta(days = 1)
		
		print('looping through survey entries in arcgis online and pinging the i-Tree API for each')
		for row in cursor:
			# skip entries from today and yesterday
			if row[39]>yesterday:
				api_url = 'https://dtbeapi.daveyinstitute.com/v2/getGrowoutTreeBenefit'
				
				# build the REST query by plugging in submitted (and default) values
				params = {
	'key':'272752b916654994a7e781cd037fc4ed',
	'ProjectYears':'1',
	'MortalityRate':'0.0',
	'Longitude':row[10],
	'Latitude':row[11],
	'TreeAmount':'1',
	'Species':row[13],
	'DBHCentimeter':row[17],
	'TreeHeightMeter':'-1',
	'TreeCrownHeightMeter':'-1',
	'TreeCrownWidthMeter':'-1',
	'Condition':row[21],
	'CLE':'3',
	'EnergyOnly':'0',
	'HouseVintage':row[26],
	'DirectionDegree':row[27],
	'DistanceMeter':row[28],
	'Heated':row[30],
	'AirConditioned':row[31]
	}
				
				print('params: '+params)
				response = requests.get(url = api_url, params = params)
				if response.status_code == 200:
					root = ET.fromstring(response.content)
					print('200 OK')
					## TODO remove temporary - fixed in survey
					for child in root.find('.//ProjectYears'):
						row[8] = child.text
						cursor.updateRow(row)
					for child in root.findall('.//HydroBenefit/'):
						if child.tag == 'RunoffAvoided':
							row[43] = child.text
							cursor.updateRow(row)
						elif child.tag == 'RunoffAvoidedValue': 
							row[44] = child.text
							cursor.updateRow(row)
						elif child.tag == 'Interception':
							row[45] = child.text
							cursor.updateRow(row)
						elif child.tag == 'PotentialEvaporation':
							row[46] = child.text
							cursor.updateRow(row)
						elif child.tag == 'PotentialEvapotranspiration':
							row[47] = child.text
							cursor.updateRow(row)
						elif child.tag == 'Evaporation':
							row[48] = child.text
							cursor.updateRow(row)
						elif child.tag == 'Transpiration':
							row[49] = child.text
							cursor.updateRow(row)
	
					for child in root.findall('.//EnergyBenefit/'):
						if child.tag == 'MbtuHeatShade':
							row[50] = child.text
							cursor.updateRow(row)
						elif child.tag == 'MbtuHeatShadeValue':
							row[51] = child.text
							cursor.updateRow(row)
						elif child.tag == 'MbtuHeatClimate':
							row[52] = child.text
							cursor.updateRow(row)
						elif child.tag == 'MbtuHeatClimateValue':
							row[53] = child.text
							cursor.updateRow(row)
						elif child.tag == 'MbtuWindHeat':
							row[54] = child.text
							cursor.updateRow(row)
						elif child.tag == 'MbtuWindHeatValue':
							row[55] = child.text
							cursor.updateRow(row)
						elif child.tag == 'KWhElectricityHeatShade':
							row[56] = child.text
							cursor.updateRow(row)
						elif child.tag == 'KWhElectricityHeatShadeValue':
							row[57] = child.text
							cursor.updateRow(row)
						elif child.tag == 'KWhElectricityHeatClimate':
							row[58] = child.text
							cursor.updateRow(row)
						elif child.tag == 'KWhElectricityHeatClimateValue':
							row[59] = child.text
							cursor.updateRow(row)
						elif child.tag == 'KWhElectricityHeatWind':
							row[60] = child.text
							cursor.updateRow(row)
						elif child.tag == 'KWhElectricityHeatWindValue':
							row[61] = child.text
							cursor.updateRow(row)
						elif child.tag == 'KWhElectricityCoolShade':
							row[62] = child.text
							cursor.updateRow(row)
						elif child.tag == 'KWhElectricityCoolShadeValue':
							row[63] = child.text
							cursor.updateRow(row)
						elif child.tag == 'KWhElectricityCoolClimate':
							row[64] = child.text
							cursor.updateRow(row)
						elif child.tag == 'KWhElectricityCoolClimateValue':
							row[65] = child.text
							cursor.updateRow(row)
						elif child.tag == 'KWhPrice':
							row[66] = child.text
							cursor.updateRow(row)
						elif child.tag == 'ThermPrice':
							row[67] = child.text
							cursor.updateRow(row)
	
					for child in root.findall('.//AirQualityBenefit_PollRemoved/'):
						if child.tag == 'CO':
							row[68] = child.text
							cursor.updateRow(row)
						elif child.tag == 'COValue':
							row[69] = child.text
							cursor.updateRow(row)
						elif child.tag == 'NO2':
							row[70] = child.text
							cursor.updateRow(row)
						elif child.tag == 'NO2Value':
							row[71] = child.text
							cursor.updateRow(row)
						elif child.tag == 'SO2':
							row[72] = child.text
							cursor.updateRow(row)
						elif child.tag == 'SO2Value':
							row[73] = child.text
							cursor.updateRow(row)
						elif child.tag == 'O3':
							row[74] = child.text
							cursor.updateRow(row)
						elif child.tag == 'O3Value':
							row[75] = child.text
							cursor.updateRow(row)
						elif child.tag == 'PM25':
							row[76] = child.text
							cursor.updateRow(row)
						elif child.tag == 'PM25Value':
							row[77] = child.text
							cursor.updateRow(row)
							
					for child in root.findall('.//AirQualityBenefit_PollAvoided/'):
						if child.tag == 'CO':
							row[78] = child.text
							cursor.updateRow(row)
						elif child.tag == 'COValue':
							row[79] = child.text
							cursor.updateRow(row)
						elif child.tag == 'NO2':
							row[80] = child.text
							cursor.updateRow(row)
						elif child.tag == 'NO2Value':
							row[81] = child.text
							cursor.updateRow(row)
						elif child.tag == 'SO2':
							row[82] = child.text
							cursor.updateRow(row)
						elif child.tag == 'SO2Value':
							row[83] = child.text
							cursor.updateRow(row)
						elif child.tag == 'PM25':
							row[84] = child.text
							cursor.updateRow(row)
						elif child.tag == 'PM25Value':
							row[85] = child.text
							cursor.updateRow(row)
						elif child.tag == 'VOC':
							row[86] = child.text
							cursor.updateRow(row)
						elif child.tag == 'VOCValue':
							row[87] = child.text
							cursor.updateRow(row)
			        
					for child in root.findall('.//CO2Benefit_Sequestered/'):
						if child.tag == 'CO2':
							row[88] = child.text
							cursor.updateRow(row)
						elif child.tag == 'CO2Value':
							row[89] = child.text
							cursor.updateRow(row)
			        
					for child in root.findall('.//CO2Benefit_Avoided/'):
						if child.tag == 'CO2':
							row[90] = child.text
							cursor.updateRow(row)
						elif child.tag == 'CO2Value':
							row[91] = child.text
							cursor.updateRow(row)
			
					for child in root.findall('.//TreeCarbon/'):
						if child.tag == 'CarbonStorage':
							row[92] = child.text
							cursor.updateRow(row)
						elif child.tag == 'CarbonDioxideStorage':
							row[93] = child.text
							cursor.updateRow(row)
						elif child.tag == 'CarbonDioxideStorageValue':
							row[94] = child.text
							cursor.updateRow(row)
						elif child.tag == 'DryWeight':
							row[95] = child.text
							cursor.updateRow(row)
				elif response.status_code != 200:
					print('There was a problem with the query: '+response.status_code)
					
		print('data on arcgis online has been updated with i-Tree data')
	
		# finally, run calculations on fields updated above and return calculated values to 'Calc_' fields (functions take Field Name in the schema)
		print('running calculations...')
		arcpy.management.CalculateField(arcgis_url, 'Calc_CarbonRemovalValue', '!Sequestered_CO2Value!'+'!Avoided_CO2Value!', 'PYTHON3')
		arcpy.management.CalculateField(arcgis_url, 'Calc_PollRemoved','!PollRemoved_CO! + !PollRemoved_NO2! + !PollRemoved_SO2! + !PollRemoved_O3! + !PollRemoved_PM25!', 'PYTHON3')
		arcpy.management.CalculateField(arcgis_url, 'Calc_PollRemovedValue','!PollRemoved_COValue! + !PollRemoved_NO2Value! + !PollRemoved_SO2Value! + !PollRemoved_O3Value! + !PollRemoved_PM25Value!', 'PYTHON3')
		arcpy.management.CalculateField(arcgis_url, 'Calc_RunoffAvoidedGallons','!RunoffAvoided! * 264.172', 'PYTHON3')
		arcpy.management.CalculateField(arcgis_url, 'Calc_EnergySavings','!KWhElectricityHeatClimateValue'  +  '!KWhElectricityCoolClimateValue' + '!MbtuHeatClimateValue', 'PYTHON3')
		print('fields have been calculated')
		print('done! go hug a tree')
except Exception as e:
	print(e)

Knox Co Community Trees
https://services2.arcgis.com/NTgkB9vlJK9YfRbY/arcgis/rest/services/Knox_Co_Community_Trees/FeatureServer/0
looping through survey entries in arcgis online and pinging the i-Tree API for each
data on arcgis online has been updated with i-Tree data
running calculations...
fields have been calculated
done! hug a tree


<class 'TypeError'>: unsupported operand type(s) for +: 'float' and 'NoneType'